In [36]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import os
from pyarrow import HadoopFileSystem

/var/folders/bz/0mv11kbs3fz_x4xp7yfflz940000gn/T/ipykernel_7002/1103317066.py:7: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [37]:
def fetch_from_to_date(token,delta=14):
    with sqlite3.connect('data/data.db') as conn:
        cur = conn.cursor()
        cur.execute(f"SELECT ts,token FROM ohlc WHERE token='{token}'")
        data = cur.fetchall()
        data = pd.DataFrame(data,columns=['ts','token'])
    date = data.ts.str[:10].unique()
    for i in range(1,len(date)):
        s = date[-i].split('-')
        if datetime(int(s[0]),int(s[1]),int(s[2])).weekday()==6:
            to_date = datetime(int(s[0]),int(s[1]),int(s[2]))
            break
    
    return (to_date-timedelta(delta),to_date)


In [38]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [39]:
with sqlite3.connect('data/data.db') as conn:
    cur = conn.cursor()
    cur.execute(f"SELECT token,ts FROM ohlc")
    data = cur.fetchall()
    data = pd.DataFrame(data,columns=['token','ts'])

tokens = data.token.unique().tolist()
connection_string = 'sqlite:///data/data.db'

In [40]:
for token in tokens:
    from_date,to_date = fetch_from_to_date(token)
    print(f'running for {token} for date {from_date} to {to_date}',end='\r')
    res = pm.execute_notebook(
    'template.ipynb',
    f'{base_output_folder}/market_analysis_{token}_{from_date}.ipynb',
    parameters = {
        'from_date': f'{from_date}',
        'to_date': f'{to_date}',
        'token': token,
        'connection_string': connection_string,
    }
)

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

In [41]:
nbs = sb.read_notebooks('outputs')

In [42]:
def sum_by_token(scrap):
    ans = pd.Series({'token':scrap['token'].data,
                     'from_date':scrap['from_date'].data,
                     'to_date':scrap['to_date'].data,
                     'hour_ret':scrap['prices'].data.h_ret.mean(),
                     'volatility': scrap['prices'].data.volatility.mean(),
                     'MDD':scrap['prices'].data.MDD.iloc[-1],
                     'last-first':scrap['prices'].data.close.iloc[-1]-scrap['prices'].data.close.iloc[0]})
    return ans
                
summary = pd.DataFrame([sum_by_token(scrap) for book, scrap in nbs.notebook_scraps.items()])   

In [43]:
summary

,token,from_date,to_date,hour_ret,volatility,MDD,last-first
0,AAVE,2021-11-28 00:00:00,2021-12-12 00:00:00,-7.966076e-04,0.216086,-0.378718,-54.6270
1,ADA,2021-11-28 00:00:00,2021-12-12 00:00:00,-3.838681e-04,0.216895,-0.312827,-0.1864
2,ATOM,2021-11-28 00:00:00,2021-12-12 00:00:00,-2.990623e-04,0.301824,-0.329570,-2.5480
3,AVAX,2021-11-28 00:00:00,2021-12-12 00:00:00,-8.466182e-04,0.296780,-0.385790,-28.0900
4,BTC,2021-11-28 00:00:00,2021-12-12 00:00:00,-3.406668e-04,0.156635,-0.206462,-5905.8800
5,COMP,2021-11-28 00:00:00,2021-12-12 00:00:00,-9.904995e-04,0.228941,-0.377871,-76.5000
6,CRV,2021-11-28 00:00:00,2021-12-12 00:00:00,-8.274572e-04,0.334854,-0.394862,-1.1723
7,DOT,2021-11-28 00:00:00,2021-12-12 00:00:00,-7.070815e-04,0.251924,-0.350800,-7.4170
8,ETH,2021-11-28 00:00:00,2021-12-12 00:00:00,-5.362780e-05,0.177805,-0.188101,-72.9700
9,SOL,2021-11-28 00:00:00,2021-12-12 00:00:00,-3.683968e-04,0.248157,-0.317207,-22.4020
